### Libraries

In [1]:
import glob
import os
import pandas as pd

___
### Выгрузка данных 

In [2]:
survey_fcasts = pd.DataFrame()

file_type = 'csv'
seperator =','

for f in glob.glob("*." + file_type):
    if f[:13] == 'survey_fcasts':
        data = pd.read_csv(f, sep=seperator, low_memory=False, encoding='ISO-8859-1')
        survey_fcasts = pd.concat([survey_fcasts, data])
    

ifps = pd.read_csv('ifps.csv', sep=seperator, encoding='ISO-8859-1')

In [3]:
survey_fcasts.head(5)

,ifp_id,ctt,cond,training,team,user_id,forecast_id,fcast_type,answer_option,value,fcast_date,expertise,q_status,viewtime,year,timestamp
0,1004-0,1a,1,a,NaN,600.0,-200987.0,0,a,0.10,2011-08-31,1.0,closed,NaN,1,2011-08-31 16:17:18
1,1004-0,1a,1,a,NaN,600.0,-200987.0,0,b,0.90,2011-08-31,1.0,closed,NaN,1,2011-08-31 16:17:18
2,1008-0,1a,1,a,NaN,600.0,-200986.0,0,a,0.25,2011-08-31,2.0,closed,NaN,1,2011-08-31 16:18:27
3,1008-0,1a,1,a,NaN,600.0,-200986.0,0,b,0.75,2011-08-31,2.0,closed,NaN,1,2011-08-31 16:18:27
4,1006-0,1a,1,a,NaN,600.0,-200985.0,0,a,0.10,2011-08-31,3.0,closed,NaN,1,2011-08-31 16:18:47


In [4]:
ifps.head(5)

,ifp_id,q_type,q_text,q_desc,q_status,date_start,date_suspend,date_to_close,date_closed,outcome,short_title,days_open,n_opts,options
0,1001-0,0,"Will the Six-Party talks (among the US, North ...",'In' refers to any time during the remainder o...,closed,9/1/11,12/30/11 0:00,12/31/11,1/2/12,b,Six-party talks resume,123.0,2,"(a) Yes, (b) No"
1,1002-0,0,Who will be inaugurated as President of Russia...,'In' refers to any time during the 2012 calend...,closed,9/1/11,5/14/12 0:00,5/15/12,5/6/12,b,president of Russia,248.0,3,"(a) Medvedev, (b) Putin, (c) Neither"
2,1003-0,0,Will Serbia be officially granted EU candidacy...,A 'yes' answer to this question requires not o...,closed,9/1/11,12/30/11 0:00,12/31/11,1/3/12,b,Serbia EU candidacy,124.0,2,"(a) Yes, (b) No"
3,1004-0,0,Will the United Nations General Assembly recog...,'By' means at or prior to the end of the day o...,closed,9/1/11,9/29/11 0:00,9/30/11,9/30/11,b,UN-GA recognize Palestine,29.0,2,"(a) Yes, (b) No"
4,1005-0,0,Will Daniel Ortega win another term as Preside...,If the Nicaraguan elections do not occur in la...,closed,9/1/11,11/4/11 0:00,11/5/11,11/5/11,a,Ortega win in Nicaragua,65.0,2,"(a) Yes, (b) No"


In [5]:
q_ifps = pd.DataFrame(ifps['q_text'].unique(), columns=['questions'])
unique_q_num = q_ifps.nunique()

# show some of them
(
q_ifps.questions[0], 
q_ifps.questions[391], 
q_ifps.questions[200] 
)

('Will the Six-Party talks (among the US, North Korea, South Korea, Russia, China, and Japan) formally resume in 2011?',
 'Will **national military force(s) from one or more countries *intervene in Syria **before 1 December 2014?',
 'When will Mariano Rajoy vacate the office of Prime Minister of Spain?')

In [6]:
f'Всего вопросов : {unique_q_num[0]}'

'Всего вопросов : 527'

___
### Предсказание людей в зависимости от их ответа в прошлый год

In [72]:
user_answers = survey_fcasts[['ifp_id','user_id','q_status','year','answer_option']]
user_answers = user_answers[~user_answers.user_id.isna()].reset_index(drop=True)

assert user_answers.user_id.apply(lambda x: x.is_integer()).sum() == user_answers.shape[0]

user_answers['user_id'] = user_answers['user_id'].astype(int)

In [73]:
question_answers = ifps[ifps.n_opts == 2][['ifp_id','outcome']]
assert question_answers.ifp_id.nunique() == question_answers.shape[0]

In [77]:
fit_data = user_answers.merge(question_answers, on='ifp_id', how='inner').sort_values('ifp_id')

In [104]:
fit_data

,ifp_id,user_id,q_status,year,answer_option,outcome
21314,1001-0,715,closed,1,a,b
17154,1001-0,3057,closed,1,a,b
17155,1001-0,3057,closed,1,b,b
17156,1001-0,4020,closed,1,a,b
17157,1001-0,4020,closed,1,b,b
...,...,...,...,...,...,...
1275535,6413-0,9030,closed,4,b,b
1275536,6413-0,22295,closed,4,a,b
1275537,6413-0,22295,closed,4,b,b
1274334,6413-0,3337,closed,4,a,b


In [100]:
fit_data[(fit_data.ifp_id == '1338-1') | (fit_data.ifp_id == '1338-2')]

,ifp_id,user_id,q_status,year,answer_option,outcome
932599,1338-1,15335,voided,3,b,NaN
932627,1338-1,8190,voided,3,b,NaN
932626,1338-1,8190,voided,3,a,NaN
932625,1338-1,8385,voided,3,b,NaN
932624,1338-1,8385,voided,3,a,NaN
...,...,...,...,...,...,...
932896,1338-2,1201,voided,3,a,NaN
932895,1338-2,6688,voided,3,b,NaN
932894,1338-2,6688,voided,3,a,NaN
932893,1338-2,817,voided,3,b,NaN


In [103]:
fit_data[fit_data.ifp_id.apply(lambda x: x[:4] == '1338')].ifp_id.unique()

array(['1338-1', '1338-2'], dtype=object)